##Using Plotly to render a covariance ellipse
This is the code used in later notebooks to render ellipses for robot/marker uncertainties

In [10]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

#function to return the x and y coordinates of points on an ellipse
#specified by pos (centre) and covariance
def ellipse(pos, covariance, numpts=20, scale=1):
  a = covariance[0,0]
  b = covariance[0,1]
  c = covariance[1,1]

  # we need the square root of the covariance - this magic computes sqrt
  # such that sqrt multiplied by its transpose is covariance
  cp = np.sqrt(c)
  bp = b/cp
  ap = np.sqrt(a-bp*bp)
  sqrt=np.array([[ap,bp],[0,cp]])

  # start with empty arrays for x and y coords and populate them
  xs,ys=[],[]
  for i in range(numpts+1):
    theta = i*2*np.pi/numpts
    circlepos = np.array([scale*np.cos(theta),scale*np.sin(theta)])
    coord = np.matmul(sqrt,circlepos)+pos
    xs.append(coord[0])
    ys.append(coord[1])
  #xs.append(None)
  #ys.append(None)
  return (xs,ys)

# render an animated sequence of ellipses
cov = np.eye(2)
centre = np.array([0,0])

x,y,a=[],[],[]
for i in range(-9,10):
  xycov=i*0.1
  cov[0,1]=xycov
  cov[1,0]=xycov
  xs,ys=ellipse(centre,cov,numpts=100)
  x+=xs
  y+=ys
  a+=[xycov]*len(xs) # this is the value to animate on

fig = px.line(x=x,y=y,animation_frame=a)
fig.update_layout(sliders=[{"currentvalue": {"prefix": "XY Covariance="}}])
fig.update_xaxes(showgrid=False, range = [-1.5,1.5])
fig.update_yaxes(showgrid=False, range = [-1.5,1.5])
fig.update_layout(template="plotly_white", showlegend=False)
fig.update_layout(width = 500, height=530) #makes it look square on my screen
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 100
#fig.update_layout(texttemplate='%{percent:.1%f}')
fig.show()